# 7. fejezet: Chat alkalmazások készítése
## Github Models API Gyorsindítás

Ez a jegyzet az [Azure OpenAI Minták Tárolójából](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) származik, amely olyan jegyzeteket tartalmaz, amelyek hozzáférnek az [Azure OpenAI](notebook-azure-openai.ipynb) szolgáltatásokhoz.


# Áttekintés  
„A nagy nyelvi modellek olyan függvények, amelyek szöveget alakítanak át szöveggé. Ha megadunk egy szöveges bemenetet, a nagy nyelvi modell megpróbálja megjósolni, mi lesz a következő szöveg”(1). Ez a „gyors kezdés” jegyzetfüzet bevezetést nyújt a felhasználóknak a LLM-ek főbb fogalmaiba, az AML használatához szükséges alapvető csomagokba, egy könnyed bevezetést az utasítások megtervezésébe, valamint néhány rövid példát különböző felhasználási területekre.


## Tartalomjegyzék  

[Áttekintés](../../../../07-building-chat-applications/python)  
[Az OpenAI szolgáltatás használata](../../../../07-building-chat-applications/python)  
[1. Az OpenAI szolgáltatás létrehozása](../../../../07-building-chat-applications/python)  
[2. Telepítés](../../../../07-building-chat-applications/python)    
[3. Hitelesítő adatok](../../../../07-building-chat-applications/python)  

[Felhasználási esetek](../../../../07-building-chat-applications/python)    
[1. Szöveg összefoglalása](../../../../07-building-chat-applications/python)  
[2. Szöveg osztályozása](../../../../07-building-chat-applications/python)  
[3. Új terméknevek generálása](../../../../07-building-chat-applications/python)  
[4. Osztályozó finomhangolása](../../../../07-building-chat-applications/python)  

[Hivatkozások](../../../../07-building-chat-applications/python)


### Készítsd el az első promptodat  
Ez a rövid gyakorlat alapvető bevezetést nyújt ahhoz, hogyan küldhetsz be promptokat egy modellnek a Github Models-ben egy egyszerű feladathoz, például az "összefoglaláshoz".

**Lépések**:  
1. Telepítsd az `azure-ai-inference` könyvtárat a python környezetedbe, ha még nem tetted meg.  
2. Töltsd be a szokásos segédkönyvtárakat, és állítsd be a Github Models hitelesítést.  
3. Válassz egy modellt a feladatodhoz  
4. Készíts egy egyszerű promptot a modell számára  
5. Küldd el a kérésedet a modell API-nak!


### 1. Telepítse az `azure-ai-inference` csomagot


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. A megfelelő modell kiválasztása  
A GPT-3.5-turbo vagy a GPT-4 modellek képesek megérteni és előállítani természetes nyelvet.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Prompttervezés  

„A nagy nyelvi modellek varázsa abban rejlik, hogy amikor hatalmas mennyiségű szövegen tanulják meg minimalizálni az előrejelzési hibát, a modellek végül olyan fogalmakat sajátítanak el, amelyek hasznosak ezekhez az előrejelzésekhez. Például megtanulják az olyan fogalmakat, mint”(1):

* hogyan kell helyesen írni
* hogyan működik a nyelvtan
* hogyan lehet átfogalmazni
* hogyan kell kérdésekre válaszolni
* hogyan kell beszélgetést folytatni
* hogyan kell több nyelven írni
* hogyan kell programozni
* stb.

#### Hogyan lehet irányítani egy nagy nyelvi modellt  
„A nagy nyelvi modellek bemenetei közül messze a legnagyobb hatással a szöveges prompt van”(1).

A nagy nyelvi modelleket többféleképpen is lehet promptolni, hogy választ adjanak:

Instrukció: Mondd el a modellnek, mit szeretnél
Kiegészítés: Késztessük a modellt arra, hogy befejezze azt, amit elkezdtünk
Demonstráció: Mutassuk meg a modellnek, mit szeretnénk, akár:
Néhány példával a promptban
Több száz vagy ezer példával egy finomhangolt tanító adathalmazban



#### Három alapvető irányelv van a promptok készítéséhez:

**Mutasd meg és mondd el**. Tedd egyértelművé, mit szeretnél, akár utasításokkal, példákkal, vagy ezek kombinációjával. Ha azt szeretnéd, hogy a modell egy listát ábécé sorrendbe rendezzen, vagy egy bekezdést érzelem szerint osztályozzon, mutasd meg neki, hogy ezt várod el.

**Adj minőségi adatokat**. Ha osztályozót szeretnél készíteni, vagy azt akarod, hogy a modell kövessen egy mintát, gondoskodj róla, hogy elegendő példa álljon rendelkezésre. Mindig ellenőrizd a példáidat — a modell általában elég okos ahhoz, hogy átlásson az alapvető helyesírási hibákon és választ adjon, de előfordulhat, hogy szándékosnak veszi ezeket, és ez befolyásolhatja a választ.

**Ellenőrizd a beállításokat.** A temperature és top_p beállítások szabályozzák, mennyire determinisztikus a modell válasza. Ha olyan választ vársz, amire csak egy helyes megoldás van, érdemes ezeket alacsonyabbra állítani. Ha változatosabb válaszokat szeretnél, akkor magasabbra is állíthatod. A leggyakoribb hiba ezekkel a beállításokkal kapcsolatban, hogy sokan azt hiszik, ezek a „leleményesség” vagy „kreativitás” szabályozói.


Forrás: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Szöveg összefoglalása  
#### Feladat  
Foglalja össze a szöveget úgy, hogy a végére hozzáad egy 'tl;dr:'-t. Figyelje meg, hogy a modell képes számos feladatot elvégezni további utasítások nélkül is. Kísérletezhet részletesebb promptokkal is, hogy módosítsa a modell viselkedését és testre szabja az összefoglalást(3).  

A közelmúltban végzett kutatások jelentős előrelépést mutattak számos NLP feladatban és mérőszámban, amikor egy nagy szövegkorpusz előzetes betanítását követően egy adott feladatra finomhangolást végeztek. Bár ezek a módszerek általában feladatfüggetlenek az architektúra szempontjából, mégis szükségük van több ezer vagy akár több tízezer példából álló, feladatspecifikus finomhangolási adathalmazokra. Ezzel szemben az emberek általában már néhány példából vagy egyszerű utasításokból is képesek új nyelvi feladatokat megoldani – ez az, amivel a jelenlegi NLP rendszerek még mindig nehezen birkóznak meg. Itt bemutatjuk, hogy a nyelvi modellek méretének növelése jelentősen javítja a feladatfüggetlen, kevés példán alapuló teljesítményt, néha még a korábbi csúcsteljesítményű finomhangolási megközelítésekkel is versenyképes szintet ér el.



Tl;dr


# Gyakorlatok különböző felhasználási esetekre  
1. Szöveg összefoglalása  
2. Szöveg osztályozása  
3. Új terméknevek generálása


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Szöveg osztályozása  
#### Feladat  
Oszd be az elemeket olyan kategóriákba, amelyeket a lekérdezés során adunk meg. Az alábbi példában mind a kategóriákat, mind az osztályozandó szöveget megadjuk a promptban (*playground_reference).

Ügyfél érdeklődése: Üdvözlöm, az egyik billentyű a laptopom billentyűzetén nemrég eltört, és szükségem lenne egy cserére:

Osztályozott kategória:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Új terméknevek generálása
#### Kihívás
Készíts termékneveket példaszavakból! Az utasításban megadjuk, hogy milyen termékhez kell neveket alkotni. Egy hasonló példát is mellékelünk, hogy lásd, milyen mintát várunk. A temperature értéket magasra állítottuk, hogy a válaszok véletlenszerűbbek és kreatívabbak legyenek.

Termékleírás: Otthoni turmixkészítő
Kulcsszavak: gyors, egészséges, kompakt.
Terméknevek: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Termékleírás: Egy pár cipő, amely bármilyen lábmérethez igazodik.
Kulcsszavak: alkalmazkodó, illeszkedő, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Hivatkozások  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio példák](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Legjobb gyakorlatok a GPT-3 finomhangolásához szöveg osztályozására](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# További segítségért  
[OpenAI Kereskedelmi Csapat](AzureOpenAITeam@microsoft.com)


# Közreműködők
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Jogi nyilatkozat**:  
Ez a dokumentum AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti, eredeti nyelvű dokumentum tekintendő hiteles forrásnak. Kritikus információk esetén javasoljuk a professzionális, emberi fordítást. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
